<a href="https://colab.research.google.com/github/Yk-rin04/Prog2kakushin/blob/main/work2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Work2
numpyを用いたプログラム

音の名前（A,B,C等），音の長さ，どのようにするか（連結または重ね合わせ）を入力することによって作曲ができるプログラム．